# Введение в NLP, часть 2


In [1]:
!pip install -U pip
!pip install transformers datasets torch seqeval evaluate

  Using cached pip-24.2-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-24.2-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 54.4 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=3100f88017f734b2a682a27372a6c8db4bbeb0fccc28bc085d76746a172c63b0
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 


## NER c BERT (25 баллов)

1. Взять датасет из предыдущего ДЗ и обучить на нём BERT.
2. Обучить BERT на подготовленном датасете
3. Оценить результат, сравнить с моделью из первого ДЗ

### Подготовка данных (5 баллов)

Подумать о:
1) Как subword токенизация повлияет на BIO раззметку?
2) Что делать с `[CLS]` и `[SEP]` токенами? (Проверьте что использует `DataCollatorForTokenClassification`)

> Hint! Токенайзер умеет работать с предразделёнными на "слова" текстами

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer


BASE_NER_MODEL = "bert-base-cased"
bert_tokenizer = AutoTokenizer.from_pretrained(BASE_NER_MODEL)


conll2003 = load_dataset("conll2003", trust_remote_code=True)
conll2003

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

...

README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [ ]:
example = conll2003["train"][100]
example

{'id': '100',
 'tokens': ['Rabinovich',
  'is',
  'winding',
  'up',
  'his',
  'term',
  'as',
  'ambassador',
  '.'],
 'pos_tags': [21, 42, 39, 33, 29, 21, 15, 21, 7],
 'chunk_tags': [11, 21, 22, 15, 11, 12, 13, 11, 0],
 'ner_tags': [1, 0, 0, 0, 0, 0, 0, 0, 0]}

* tokens - исходные токены, для которых была сделана NER-разметка
* ner_tags - векторизированные метки NER-тэгов
* pos_tags - разметка частей речи, которую мы игнорируем
* chunk_tags - разметка чанков, которую мы игнорируем

Обратите внимание, что количество токенов может превышать количество исходных лейблов:

In [ ]:
bert_tokenizer(example["tokens"], is_split_into_words=True).tokens()

['[CLS]',
 'Ra',
 '##bino',
 '##vich',
 'is',
 'winding',
 'up',
 'his',
 'term',
 'as',
 'ambassador',
 '.',
 '[SEP]']

Значение тэга в ner_tags отображается в метку NER:

In [ ]:
print("NER TAGS", example["ner_tags"])
print(conll2003["train"].features["ner_tags"].feature)

NER TAGS [1, 0, 0, 0, 0, 0, 0, 0, 0]
ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None)


In [ ]:
print("Оригинальные токены")
print(example["tokens"])
print("Векторизированные NER метки токенов")
print(example["ner_tags"])
tags_str = []
features = conll2003["train"].features["ner_tags"].feature
for tag in example["ner_tags"]:
    tags_str.append(features.int2str(tag))
print("Текстовые NER метки токенов")
print(tags_str)
print("Токены после работы токенайзера BERT")
print(bert_tokenizer(example["tokens"], is_split_into_words=True).tokens())

Оригинальные токены
['Rabinovich', 'is', 'winding', 'up', 'his', 'term', 'as', 'ambassador', '.']
Векторизированные NER метки токенов
[1, 0, 0, 0, 0, 0, 0, 0, 0]
Текстовые NER метки токенов
['B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Токены после работы токенайзера BERT
['[CLS]', 'Ra', '##bino', '##vich', 'is', 'winding', 'up', 'his', 'term', 'as', 'ambassador', '.', '[SEP]']


Вспомним немного, как работают метки в задаче мер в кодировке BIO. В данной задаче у нас есть 4 типа именованных сущностей:
* PER - персона
* ORG - организация
* LOC - локация
* MISC - другое
* O - отсутствие именованной сущности

У каждого типа именованных 2 префикса:
* `B-` - beginning, т.е. начало именованной сущности.
* `I-` - inside, т.е. продолжение ранее начатой именованной сущностью.

В исходной токенизации

`['Rabinovich', 'is', 'winding', 'up', 'his', 'term', 'as', 'ambassador', '.']`
метки выглядят как

`['B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']`
т.е. `Rabinovich` является персоной. На следующем токене именованная сущность заканчивается, т.к. у него метка `O`.

После токенизации BERT наш сэмпл превращается в следующие токены:

`['[CLS]', 'Ra', '##bino', '##vich', 'is', 'winding', 'up', 'his', 'term', 'as', 'ambassador', '.', '[SEP]']`
Обратим внимание, что один токен `Rabinovich` с меткой `B-PER` был разбит токенизатором берта на 3 токена: `'Ra', '##bino', '##vich'`. Им нужно поставить в соответствие 3 метки: `B-PER, I-PER, I-PER`, т.е. мы разбиваем метку исходного токена на новые токены.

Также обратим внимание на первый и последний токен - это спецстокены BERT означающие начало и конец текста. Им можно дать метки `O`, т.к. они не являются частью исходного текста, но мы будем давать им особое векторизированное значение -100. В [документации pytroch](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) у кроссэнтропийной функции потерь это дефолтное значение `ignore_index`, т.е. метки, которую мы будем игнорировать. Библиотека transformers также использует это значение. Таким образом на токенах, у которых стоит -100 в качестве векторизированного NER-тэга, не будет происходить обучение, они будут проигнорированы.




Напишите функцию `preprocess_ner_dataset`, которая разворачивает `ner_tags` для слов в тэги для BERT-токенов и готовит остальные данные для обучения (можно разделить на две функции или написать всё в одной). В резултате применения `conll2003.map(preprocess_ner_dataset)`, в каждом примере:
1. Добавляется токенизированный вход (`input_ids`, `token_type_ids` и `attention_mask`). При конструировании этих векторов вручную нужно проставить `attention_mask` полностью единицами, т.к. в паддинги в сэмплах появляются только в рамках батчей, а `token_type_ids` полностью нулями.
2. `ner_tags` разворачивается в `labels` для входных токенов

Что можно использовать:
* у объекта `conll2003["train"].features["ner_tags"].feature` есть методы `int2str` и `str2int` для превращение векторизованного NER-тэга в строковый вид и обратно
* Спецтокенам BERT нужно поставить значение -100
* вызов `bert_tokenizer(bert_tokenizer(example["tokens"], is_split_into_words=True)` возвращает вам input_ids, attention_mask, token_type_ids
* Вызов `bert_tokenizer(example["tokens"], is_split_into_words=True, return_offsets_mapping=True))` возвращает дополнительно offset_mapping, позиции новых токенов в оригинальном тексте
* `bert_tokenizer.vocab` - для превращения токенов в их индексы в словаре
* `bert_tokenizer.tokenize` - разбитие текста (в том числе и исходных токенов) на токены BERT

Ваша задача:
1. Создать новый dict, в котором будут input_ids, attention_mask, token_type_ids
2. Добавить в него labels - векторизированные NER-тэги, которые будут разбиты в соответствии с токенизацией BERT. Для этого можно можно разбить каждый токен отдельно и размножить его метки. Альтернативно можно использовать информацию об оффсетах токенов BERT, чтобы понять, частью какого исходного токена и какой исходной метки является данный BERT-токен.

In [ ]:
def preprocess_ner_dataset(examples):
    # Проверяем, является ли входной пример батчем или одиночным примером
    is_batched = isinstance(examples["tokens"][0], list)

    if is_batched:
        # Обработка батча
        tokens = examples["tokens"]
        ner_tags = examples["ner_tags"]

        # Токенизация батча данных
        tokenized_inputs = bert_tokenizer(
            tokens,
            is_split_into_words=True,
            return_offsets_mapping=True,
            truncation=True,
            padding=False
        )

        # Инициализируем список для всех меток
        all_labels = []

        for i in range(len(tokenized_inputs["input_ids"])):
            # Получаем word_ids для текущего примера в батче
            word_ids = tokenized_inputs.word_ids(batch_index=i)
            labels = ner_tags[i]

            previous_word_idx = None
            label_ids = []
            for idx, word_idx in enumerate(word_ids):
                if word_idx is None:
                    # Специальные токены
                    label_ids.append(-100)
                else:
                    label = labels[word_idx]
                    if word_idx != previous_word_idx:
                        # Начало нового слова
                        label_ids.append(label)
                    else:
                        # Субтокен текущего слова
                        if label != 0:
                            # Если метка B- (нечетное число), преобразуем в I- (четное число)
                            if label % 2 == 1:
                                label_ids.append(label + 1)
                            else:
                                label_ids.append(label)
                        else:
                            label_ids.append(0)
                    previous_word_idx = word_idx
            all_labels.append(label_ids)

        # Добавляем метки в токенизированные данные
        tokenized_inputs["labels"] = all_labels

        # Удаляем offset_mapping, чтобы он не присутствовал в выходных данных
        tokenized_inputs.pop("offset_mapping", None)

        return tokenized_inputs
    else:
        # Обработка одиночного примера
        tokens = examples["tokens"]
        ner_tags = examples["ner_tags"]

        # Токенизация одиночного примера
        tokenized_inputs = bert_tokenizer(
            tokens,
            is_split_into_words=True,
            return_offsets_mapping=True,
            truncation=True,
            padding=False
        )

        # Получаем word_ids для текущего примера
        word_ids = tokenized_inputs.word_ids()
        labels = ner_tags

        previous_word_idx = None
        label_ids = []
        for idx, word_idx in enumerate(word_ids):
            if word_idx is None:
                # Специальные токены
                label_ids.append(-100)
            else:
                label = labels[word_idx]
                if word_idx != previous_word_idx:
                    # Начало нового слова
                    label_ids.append(label)
                else:
                    # Субтокен текущего слова
                    if label != 0:
                        # Если метка B- (нечетное число), преобразуем в I- (четное число)
                        if label % 2 == 1:
                            label_ids.append(label + 1)
                        else:
                            label_ids.append(label)
                    else:
                        label_ids.append(0)
                previous_word_idx = word_idx

        # Добавляем метки в токенизированные данные
        tokenized_inputs["labels"] = label_ids

        # Удаляем offset_mapping, чтобы он не присутствовал в выходных данных
        tokenized_inputs.pop("offset_mapping", None)

        return tokenized_inputs


Пример получившегося выхода:
```python
>>> preprocessed_ner_dataset["train"][100]
{'id': '100',
 'tokens': ['Rabinovich',
  'is',
  'winding',
  'up',
  'his',
  'term',
  'as',
  'ambassador',
  '.'],
 'pos_tags': [21, 42, 39, 33, 29, 21, 15, 21, 7],
 'chunk_tags': [11, 21, 22, 15, 11, 12, 13, 11, 0],
 'ner_tags': [1, 0, 0, 0, 0, 0, 0, 0, 0],
 'input_ids': [101,
  16890,
  25473,
  11690,
  1110,
  14042,
  1146,
  1117,
  1858,
  1112,
  9088,
  119,
  102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [-100, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, -100]}
```

### Тесты

In [ ]:
processed_example = preprocess_ner_dataset(example)
required_keys = ["input_ids", "labels", "attention_mask", "token_type_ids"]
for k in required_keys:
    assert k in processed_example, f"Отсутствует поле {k}"

required_keys_set = set(required_keys)
for k in processed_example.keys():
    assert k in required_keys_set, f"В примере лишнее поле {k}"

In [ ]:
from tqdm import tqdm
for idx, example in tqdm(enumerate(conll2003["train"])):
    input_ids_real = bert_tokenizer(example["tokens"], is_split_into_words=True)["input_ids"]
    input_ids_ours = preprocess_ner_dataset(example)["input_ids"]
    assert input_ids_real == input_ids_ours, f"Ошибка токенизации на примере {idx}"
    if idx >= 100:
        break
print("Токенизация верна!")

100it [00:00, 1635.88it/s]

Токенизация верна!


In [ ]:
example = conll2003["train"][100]
processed_example = preprocess_ner_dataset(example)

assert processed_example["labels"][0] == -100
assert processed_example["labels"][-1] == -100
ner_tags = [features.int2str(i) for i in processed_example["labels"][1:-1]]
assert ner_tags == ['B-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

In [ ]:
example = conll2003["train"][200]
processed_example = preprocess_ner_dataset(example)

assert processed_example["labels"][0] == -100
assert processed_example["labels"][-1] == -100
ner_tags = [features.int2str(i) for i in processed_example["labels"][1:-1]]
assert ner_tags == ['B-ORG', 'I-ORG', 'I-ORG', 'I-ORG']

###  Возьмем для экономии 10% данных

In [ ]:
# Функция для взятия подмножества данных
def get_small_dataset(dataset, fraction=0.1, seed=42):
    small_dataset = dataset.train_test_split(test_size=1 - fraction, seed=seed)["train"]
    return small_dataset

# Применяем функцию к каждому из разделов датасета
conll2003_small = {}
for split in ['train', 'validation', 'test']:
    conll2003_small[split] = get_small_dataset(conll2003[split], fraction=0.1)

# Преобразуем в DatasetDict
from datasets import DatasetDict
conll2003_small = DatasetDict(conll2003_small)


Подготовим `data_collator`. Это особый класс, который будет заниматься батчеванием сэмплов для обучения. Он добавит паддинги во все необходимые поля.

In [ ]:
from transformers import DataCollatorForTokenClassification


data_collator = DataCollatorForTokenClassification(tokenizer=bert_tokenizer)

### Подготовка модели (5 баллов)

Два возможных пути на этой стадии:
1. Взять [готовый класс](https://huggingface.co/transformers/v3.0.2/model_doc/auto.html#automodelfortokenclassification) модели для классификации токенов. (Этот вариант настоятельно рекомендуется)
2. Взять модель как фича экстрактор ([AutoModel](https://huggingface.co/transformers/v3.0.2/model_doc/auto.html#automodel)) и самостоятельно добавить классификационную голову. Вдохновиться можно по [ссылке](https://github.com/huggingface/transformers/blob/main/src/transformers/models/bert/modeling_bert.py#L1847-L1860). Дополнительных баллов второй вариант не принесёт.

Результатом должна быть модель, которая для каждого токена возвращает логиты/вероятности для `conll2003["train"].features["ner_tags"].feature.num_classes` классов.

> Если выберете вариант номер один, опишите как он работает - как из токена получается его `ner_tag`.

In [ ]:
# Этот класс автоматически подбирает подходящую модель для классификации токенов на основе конфигураци
from transformers import AutoModelForTokenClassification, AutoModel

# Получаем количество меток
num_labels = len(features.names)
# num_labels - количество меток NER

# Извлекаем список меток из исходного датасета
label_list = conll2003_small["train"].features["ner_tags"].feature.names

# Указываем модель, которую будем использовать (например, BERT)
model_name = "bert-base-cased"

# Загружаем токенизатор и модель для классификации токенов
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=num_labels)

# Пример использования токенизатора и модели
inputs = tokenizer("Rabinovich is winding up his term as ambassador.", return_tensors="pt")

# Прогоняем входные данные через модель
outputs = model(**inputs)

# Выводим логиты (результаты перед применением softmax)
print(outputs.logits)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tensor([[[-0.0046,  0.1038,  0.0146, -0.1923,  0.4295, -0.4004, -0.1712,
           0.0971, -0.4371],
         [ 0.0979,  0.0865, -0.0925, -0.0961,  0.3647, -0.4362,  0.1651,
           0.1888, -0.3088],
         [-0.1375,  0.1256,  0.0496, -0.1105,  0.0428, -0.0992, -0.1908,
           0.1185, -0.2343],
         [-0.1748,  0.1067,  0.0184, -0.2179,  0.2069, -0.4756, -0.2118,
           0.0507, -0.1559],
         [-0.0128,  0.4143, -0.1133,  0.0630,  0.2570, -0.2637, -0.2113,
          -0.2320, -0.4384],
         [-0.1711,  0.1408,  0.0253,  0.0431,  0.3379, -0.0485, -0.7129,
           0.0027, -0.5541],
         [ 0.2988,  0.1082,  0.0240,  0.2607,  0.2764, -0.3429, -0.2607,
          -0.3050, -0.2372],
         [ 0.2574,  0.0536, -0.1184,  0.3337,  0.2554,  0.1261, -0.3039,
           0.0330, -0.3673],
         [ 0.4392, -0.0410, -0.4832,  0.2874,  0.2261,  0.1785,  0.1690,
           0.0953, -0.7117],
         [ 0.3581, -0.4505, -0.2564,  0.0552,  0.1518, -0.0064, -0.2291,
         

In [ ]:
model.device

device(type='cuda', index=0)

In [ ]:
# Сохраняем соответствие id2label и label2id в конфигурации модели
model.config.id2label = {i: label for i, label in enumerate(label_list)}
model.config.label2id = {label: i for i, label in enumerate(label_list)}

`model.config.id2label` - используем model.config.id2label для преобразования числовых меток в строковые, что делает код более чистым и независимым от внешних переменных.

### Подготовим Метрику (5 баллов)

Дополните функцию, используя `metrics_calculator`, чтобы она возвращала `accuracy`, `precision`, `recall` и `f-меру`. `eval_predictions` - это кортеж из логитов токен классификатора и `labels`, которые мы подготовили с помощью `preprocess_ner_dataset`. Нужно:
1. Преобразовать логиты в предсказанные лейблы. Учтите, что для специальных токенов лейблов нет
2. Посчитать метрики с помощью `metrics_calculator`
3. Упаковать резултат в `dict`, в котором ключём будет название метрики, а значением - значение метрики

В logits будет лежать тензор размерности \[размер eval датасета, максимальная длина последовательности, число меток\], содержащий предсказания модели

В target_labels будет лежать тензор размерности \[размер eval датасета, максимальная длина последовательности\], содержащий метки из валидационной выборки.

Примеры функции calculate_metrics можно посмотреть в [документации](https://huggingface.co/docs/evaluate/en/transformers_integrations)

In [ ]:
import evaluate  # Импортируем библиотеку evaluate для вычисления метрик
import numpy as np

# Загружаем метрику seqeval для оценки результатов NER (Named Entity Recognition)
metrics_calculator = evaluate.load("seqeval")

def calculate_metrics(eval_predictions):
    # Распаковываем предсказанные логиты и целевые метки из eval_predictions
    logits, target_labels = eval_predictions

    # Получаем предсказанные метки, выбирая индекс с максимальным логитом для каждого токена
    predicted_labels = np.argmax(logits, axis=-1)

    # Инициализируем списки для истинных и предсказанных меток без специальных токенов
    true_labels = []
    pred_labels = []

    # Проходим по каждому набору предсказаний и меток в батче
    for pred, label in zip(predicted_labels, target_labels):
        true_label = []
        pred_label = []
        # Проходим по каждой метке в последовательности
        for p, l in zip(pred, label):
            # Игнорируем метки с значением -100 (они используются для маскировки)
            if l != -100:
                true_label.append(l)
                pred_label.append(p)
        # Добавляем обработанные метки для текущей последовательности
        true_labels.append(true_label)
        pred_labels.append(pred_label)

    # Используем словарь id2label из конфигурации модели для преобразования меток из индексов в строки
    id2label = model.config.id2label

    # Преобразуем числовые метки в строковые для истинных меток
    true_labels_str = [[id2label[l] for l in seq] for seq in true_labels]
    # Преобразуем числовые метки в строковые для предсказанных меток
    pred_labels_str = [[id2label[p] for p in seq] for seq in pred_labels]

    # Вычисляем метрики, сравнивая предсказания и истинные метки
    metrics_result = metrics_calculator.compute(predictions=pred_labels_str, references=true_labels_str)

    # Возвращаем словарь с основными метриками
    return {
        "accuracy": metrics_result.get("overall_accuracy"),        # Точность (Accuracy)
        "precision": metrics_result.get("overall_precision"),      # Точность (Precision)
        "recall": metrics_result.get("overall_recall"),            # Полнота (Recall)
        "f1": metrics_result.get("overall_f1"),                    # F1-мера (F1 Score)
    }


### Применение функции предобработки к датасету


In [ ]:
# Применяем предобработку к уменьшенному датасету
processed_datasets = conll2003_small.map(preprocess_ner_dataset, batched=True)

# Удаляем ненужные колонки
processed_datasets = processed_datasets.remove_columns(['tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'id'])


Map:   0%|          | 0/1404 [00:00<?, ? examples/s]

Map:   0%|          | 0/325 [00:00<?, ? examples/s]

Map:   0%|          | 0/345 [00:00<?, ? examples/s]

### Обучение (5 баллов)

Два возможных пути на этой стадии:

1. Использовать [Trainer](https://huggingface.co/docs/transformers/main_classes/trainer) класс из `transformers`
2. Написать свой training loop. Дополнительных баллов на этом пути нет.

Опишем подробнее первый путь, т.к. он настоятельно рекомендуется.

Нужно создать класс Trainer и TrainingArguments.
В [TrainingArguments](https://huggingface.co/docs/transformers/en/main_classes/trainer#transformers.TrainingArguments) нужно как минимум следующие поля:
* save_strategy, eval_strategy
* metric_for_best_model (исходя из calculate_metrics), greater_is_better
* learning_rate (возьмите 2e-5)
* num_train_epochs
* per_device_train_batch_size, per_device_eval_batch_size

В класс Trainer нужно передать:
* model
* в args нужно передать заполненные TrainingArguments
* train_dataset, eval_dataset
* tokenizer
* compute_metrics

После чего запустить `trainer.train()`

In [ ]:
from transformers import Trainer, TrainingArguments

# Определяем TrainingArguments
training_args = TrainingArguments(
    output_dir="./results",  # Директория для сохранения результатов
    eval_strategy="epoch",  # Выполнять оценку на каждом этапе
    save_strategy="epoch",  # Сохранять модель на каждом этапе
    metric_for_best_model="f1",  # Основная метрика для выбора лучшей модели
    greater_is_better=True,  # Увеличение метрики f1 говорит о лучшей модели
    learning_rate=2e-5,  # Уровень обучения
    num_train_epochs=3,  # Количество эпох
    per_device_train_batch_size=16,  # Размер батча на обучении
    per_device_eval_batch_size=16,  # Размер батча на оценке
    logging_dir="./logs",  # Директория для логов
    load_best_model_at_end=True,  # Загружать лучшую модель в конце обучения
)

# Теперь запускаем обучение на малой выборке
trainer = Trainer(
    model=model,  # Модель
    args=training_args,  # Параметры обучения
    train_dataset= processed_datasets["train"],  # Используем тренировочный датасет
    eval_dataset= processed_datasets["validation"],  # Используем валидационный датасет
    tokenizer=tokenizer,  # Токенизатор
    data_collator=data_collator,  # Специальный коллатор для классификации токенов
    compute_metrics=calculate_metrics,  # Функция расчета метрик
)

# Запускаем обучение
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.135170,0.958990,0.707668,0.808394,0.754685
2,No log,0.110405,0.971025,0.784539,0.870438,0.825260
3,No log,0.105059,0.972957,0.810000,0.886861,0.846690


TrainOutput(global_step=264, training_loss=0.11209685152227228, metrics={'train_runtime': 160.3443, 'train_samples_per_second': 26.268, 'train_steps_per_second': 1.646, 'total_flos': 107497756036224.0, 'train_loss': 0.11209685152227228, 'epoch': 3.0})

### Обработка результатов Результатов (5 баллов)

Подумать о:
1. Во время подготовки данных мы преобразовали BIO разметку.
2. Как обратить это преобразование с помощью токенайзера?

Провалидируйте результаты на тестовом датасете.

Восстановление меток на уровне слов:


- прозодимся по предсказанным меткам на уровне субтокенов.
- Используя word_ids, группкруем предсказания субтокенов по исходным словам.
- Для каждого слова берем метку первого субтокена. Если во время подготовки данных метка B- была преобразована в I- для субтокенов, то будет возможно восстановить метку B- для начала слова.

функция нихе

In [ ]:
# Create a copy of the small dataset for evaluation
eval_datasets = conll2003_small

# Process the evaluation dataset
eval_processed_datasets = eval_datasets.map(preprocess_ner_dataset, batched=True)

# Remove unnecessary columns but keep 'tokens'
columns_to_remove = ['pos_tags', 'chunk_tags', 'ner_tags', 'id']
available_columns = eval_processed_datasets['test'].column_names
columns_to_remove = [col for col in columns_to_remove if col in available_columns]
eval_processed_datasets = eval_processed_datasets.remove_columns(columns_to_remove)


Map:   0%|          | 0/1404 [00:00<?, ? examples/s]

Map:   0%|          | 0/325 [00:00<?, ? examples/s]

Map:   0%|          | 0/345 [00:00<?, ? examples/s]

In [ ]:
# Получаем результаты предсказания на тестовом наборе
test_results = trainer.predict(processed_datasets["test"])

# Преобразуем логиты в метки, выбирая индекс с максимальным значением логита для каждого токена
predictions = np.argmax(test_results.predictions, axis=-1)

# Получаем истинные метки (label_ids) для тестового набора
label_ids = test_results.label_ids

# Получаем исходные данные (входы) из тестового набора, включая токены
inputs = processed_datasets["test"]


In [ ]:
predictions

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 3, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
label_ids

array([[-100,    0,    0, ..., -100, -100, -100],
       [-100,    3,    0, ..., -100, -100, -100],
       [-100,    0, -100, ..., -100, -100, -100],
       ...,
       [-100,    0,    0, ..., -100, -100, -100],
       [-100,    3,    0, ..., -100, -100, -100],
       [-100,    0,    0, ..., -100, -100, -100]])

In [ ]:
def align_predictions(predictions, label_ids, inputs):
    """
    Функция align_predictions используется после получения предсказаний модели
    на тестовом наборе.
    Она позволяет корректно выровнять предсказания и истинные метки, учитывая
    субтокены, и получить метки на уровне слов.
    Это необходимо для точного вычисления метрик, таких как точность, полнота
    и F1-мера, с использованием метрики seqeval.

    """

    pred_labels = []
    true_labels = []

    # Получаем словарь, сопоставляющий индексы меток их строковым представлениям
    id2label = model.config.id2label

    # Проходим по каждому примеру в батче
    for i in range(len(predictions)):
        preds = predictions[i]  # Предсказанные метки для текущего примера
        labels = label_ids[i]   # Истинные метки для текущего примера
        tokens = inputs['tokens'][i]  # Получаем исходные токены (слова) для текущего примера

        # Повторно токенизируем исходные токены, чтобы получить информацию о принадлежности токенов к словам
        tokenized_inputs = bert_tokenizer(
            tokens,
            is_split_into_words=True,
            return_offsets_mapping=False,
            truncation=True,
            padding=False
        )
        word_ids = tokenized_inputs.word_ids()  # Получаем индексы слов для каждого токена

        prev_word_idx = None  # Инициализируем предыдущий индекс слова как None

        true_label = []  # Список для хранения истинных меток на уровне слов
        pred_label = []  # Список для хранения предсказанных меток на уровне слов

        # Проходим по каждому токену в последовательности
        for idx, word_idx in enumerate(word_ids):
            if word_idx is None:
                # Пропускаем специальные токены ([CLS], [SEP], [PAD] и т.д.)
                continue

            if word_idx != prev_word_idx:
                # Если текущий индекс слова отличается от предыдущего, значит началось новое слово
                if labels[idx] != -100:
                    # Преобразуем числовые метки в строковые, используя id2label
                    curr_label = id2label.get(labels[idx], 'O')  # Истинная метка
                    curr_pred = id2label.get(preds[idx], 'O')    # Предсказанная метка
                    # Добавляем метки в соответствующие списки
                    true_label.append(curr_label)
                    pred_label.append(curr_pred)
            # Обновляем предыдущий индекс слова
            prev_word_idx = word_idx

        # Добавляем метки для текущей последовательности в общий список
        true_labels.append(true_label)
        pred_labels.append(pred_label)

    # Возвращаем списки истинных и предсказанных меток на уровне слов для всего батча
    return true_labels, pred_labels


Проверяем столбцы в processing_datasets, чтобы убедиться в наличии «токенов».

In [ ]:
print(processed_datasets["test"].column_names)


['input_ids', 'token_type_ids', 'attention_mask', 'labels']


In [ ]:
# Используем функцию align_predictions, чтобы восстановить метки на уровне слов
true_labels, pred_labels = align_predictions(predictions, label_ids, inputs)


In [ ]:
# Compute metrics
metrics_result = metrics_calculator.compute(predictions=pred_labels, references=true_labels)

print("Результаты на тестовом наборе:")
print(f"Accuracy: {metrics_result['overall_accuracy']}")
print(f"Precision: {metrics_result['overall_precision']}")
print(f"Recall: {metrics_result['overall_recall']}")
print(f"F1: {metrics_result['overall_f1']}")

Результаты на тестовом наборе:
Accuracy: 0.9715725806451613
Precision: 0.8195615514333895
Recall: 0.8556338028169014
F1: 0.8372093023255813


Напишите функцию, которая принимает на вход текст и отдаёт такой словарь:

```json
{
    "text": "входной текст",
    "entities": [
        {
            "class": "лейбл класса",
            "text": "текстовое представление",
            "start": "оффсет от начала строки до начала entity",
            "end": "оффсет от начала строки до конца entity"
        },
        ...
    ]
}

Должно выполняться такое условие:

```python
text[entity["start"]:entity["stop"]] == entity["text"]
```

In [ ]:
def do_ner(text):
    import torch

    # Токенизируем текст с сохранением офсетов
    tokenized_inputs = bert_tokenizer(
        text,
        return_offsets_mapping=True,
        return_tensors='pt',
        truncation=True,
    )

    # Получаем офсеты
    offset_mapping = tokenized_inputs.pop("offset_mapping")[0]

    # Пропускаем через модель
    with torch.no_grad():
        outputs = model(**tokenized_inputs.to(model.device))

    # Получаем предсказания
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)[0].cpu().numpy()

    # Преобразуем индексы меток в строковые метки
    id2label = model.config.id2label
    predicted_labels = [id2label[pred] for pred in predictions]

    # Инициализируем переменные для хранения сущностей
    entities = []
    entity = None

    # Проходим по каждому токену
    for idx, (label, offset) in enumerate(zip(predicted_labels, offset_mapping)):
        start_char, end_char = offset.tolist()

        # Пропускаем специальные токены
        if start_char == end_char:
            continue

        # Если офсеты выходят за пределы текста, корректируем их
        if end_char > len(text):
            end_char = len(text)

        # Разбираем метку
        if label == 'O':
            if entity is not None:
                entities.append(entity)
                entity = None
            continue

        label_parts = label.split('-')
        if len(label_parts) != 2:
            if entity is not None:
                entities.append(entity)
                entity = None
            continue

        tag, entity_class = label_parts

        # Обработка начала сущности
        if tag == 'B':
            if entity is not None:
                entities.append(entity)
            entity = {
                "class": entity_class,
                "start": start_char,
                "end": end_char,
                "text": text[start_char:end_char]
            }
        elif tag == 'I' and entity is not None and entity["class"] == entity_class:
            # Продолжаем текущую сущность
            entity["end"] = end_char
            entity["text"] = text[entity["start"]:entity["end"]]
        else:
            # Некорректная последовательность меток, завершаем текущую сущность
            if entity is not None:
                entities.append(entity)
            entity = None

    # Добавляем последнюю сущность, если она есть
    if entity is not None:
        entities.append(entity)

    # Проверяем условие для каждой сущности
    for entity in entities:
        assert text[entity["start"]:entity["end"]] == entity["text"], \
            f"Текст сущности не соответствует офсетам: {entity}"

    # Формируем итоговый словарь
    result = {
        "text": text,
        "entities": entities
    }

    return result


In [ ]:
text = "Barack Obama was born in Hawaii."
result = do_ner(text)
result

{'text': 'Barack Obama was born in Hawaii.',
 'entities': [{'class': 'PER', 'start': 0, 'end': 12, 'text': 'Barack Obama'},
  {'class': 'LOC', 'start': 25, 'end': 31, 'text': 'Hawaii'}]}

In [ ]:
text = result['text']
entities = result['entities']

for entity in entities:
    start = entity['start']
    end = entity['end']
    entity_text = entity['text']
    text_slice = text[start:end]
    if text_slice == entity_text:
        print(f"Проверка пройдена для сущности: {entity_text}")
    else:
        print(f"Проверка не пройдена для сущности: {entity_text}")
        print(f"Ожидаемый текст: '{entity_text}', а из текста получено: '{text_slice}'")


Проверка пройдена для сущности: Barack Obama
Проверка пройдена для сущности: Hawaii


## Классификация с T5 (25 баллов) v2

Требуется дообучить [t5-small](https://huggingface.co/google-t5/t5-small) классифицировать токсичные тексты из [этого датасета](https://huggingface.co/datasets/lmsys/toxic-chat). Классификатор должен работать в стиле t5 - генерировать ответ текстом.

1. Подготовить данные для бинарной классификации
	1. Придумать префикс для задачи или взять из похожей модели
	2. Выбрать тексты для обозначения классов
2. Обучить t5-small на генерацию выбранных названия классов
3. Сравнить с модель с аналогичной предобученной моделью

### Подготовка Данных (6 баллов)

Подумать о:
1. Какой префикс выбрать для новой задачи?
2. Должен ли префикс быть понятным?
3. Как выбрать метку для класса?
4. Что будет, если метки класса целиком нет в словаре?
5. Что делать с длинными текстами?

Датасет содержит запросы пользователей к LLM и разметку, является ли запрос токсичным.

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer


BASE_T5_MODEL= "t5-small"
t5_tokenizer = AutoTokenizer.from_pretrained(BASE_T5_MODEL)


toxic_chat_dataset = load_dataset("lmsys/toxic-chat", "toxicchat0124")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.46k [00:00<?, ?B/s]

(…)ata/0124/toxic-chat_annotation_train.csv:   0%|          | 0.00/8.20M [00:00<?, ?B/s]

data/0124/toxic-chat_annotation_test.csv:   0%|          | 0.00/8.09M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Место для изучения датасета:

In [3]:
toxic_chat_dataset["train"][0]

{'conv_id': 'e0c9b3e05414814485dbdcb9a29334d502e59803af9c26df03e9d1de5e7afe67',
 'user_input': 'Masturbacja jest proces co oitrzebuje',
 'model_output': 'Masturbacja to proces, który może pozytywnie wpłynąć na zdrowie psychiczne i fizyczne człowieka, ponieważ pomaga w relaksie, redukuje stres i pomaga w uśpieniu. Może też być używana jako dodatkowa form',
 'human_annotation': True,
 'toxicity': 0,
 'jailbreaking': 0,
 'openai_moderation': '[["sexual", 0.4609803557395935], ["sexual/minors", 0.0012527990620583296], ["harassment", 0.0001862536446424201], ["hate", 0.00015521160094067454], ["violence", 6.580814078915864e-05], ["self-harm", 3.212967567378655e-05], ["violence/graphic", 1.5190824342425913e-05], ["self-harm/instructions", 1.0009921425080393e-05], ["hate/threatening", 4.4459093260229565e-06], ["self-harm/intent", 3.378846486157272e-06], ["harassment/threatening", 1.7095695739044459e-06]]'}

Нас будут интересовать колонки `"user_input"` и `"toxicity"`. Убираем ненужные колонки из датасета:

In [4]:
toxic_chat_dataset = toxic_chat_dataset.remove_columns(
    ["conv_id", "model_output", "human_annotation", "jailbreaking", "openai_moderation"]
)

![](https://production-media.paperswithcode.com/methods/new_text_to_text.jpg)

Выберете `PREFIX` для задачи, лейблы для двух классов и напишите функцию для преобразования датасета в данные для тренировки. Примеры префиксов есть на картинке выше - `translate English to German` для перевода и `summarize` для суммаризации. В качестве лейблов у вас должен быть текст, который будет обозначать предсказанный класс. Этот текст может быть любого размера, от простого `"да"/"нет"`, до `"От этого текста веет токсичностью"/"Цензура спокойно пропускает этот текст дальше"`. Подумайте в чём преимущество первого подхода перед вторым.

Важно:
1. Не забыть добавить префикс перед токенизацией входного текста
2. Лейблами во время обучения выступают уже последовательности токенов, которые мы ожидаем на выходе из декодера

Текст в токенайзер можно подавать разными способами:
1. `tokenizer(text="text")` - токенизируй текст как обычно
1. `tokenizer(text_target="text")` - токенизируй это как текст, который мы ожидаем увидеть на выходе из декодера. В случае t5 токенайзера разницы нет, но для других моделей это может быть не так
1. Другие методы можно узнать посмотрев сигнатуру метода `tokenizer.__call__`

In [ ]:
# Другие методы можно узнать посмотрев сигнатуру метода
?t5_tokenizer.__call__

In [5]:
# PREFIX - Явно указывает модели, что задача классификации токсичности.
PREFIX = "classify toxicity: "
# Ограничивает длину входного текста до 512 токенов.
MAX_LENGTH = 512

# словарь из индексов классов в выбранные лейблы
# Преобразует числовые метки в текстовые значения, которые модель будет генерировать.
id2label = {
    0: "non-toxic",
    1: "toxic",
}

label2id = {v: k for k, v in id2label.items()}


def preprocess_dataset(example):
    # Добавляем префикс к пользовательскому вводу
    input_text = PREFIX + example['user_input']

    # Токенизация входного текста без паддинга
    encoding = t5_tokenizer(
        input_text,
        truncation=True,
        max_length=MAX_LENGTH,
        return_tensors='pt'
    )

    # Преобразование метки в текстовую форму
    label_text = id2label[example['toxicity']]

    # Токенизация метки с использованием text_target
    encoding_labels = t5_tokenizer(
        text_target=label_text,
        truncation=True,
        max_length=MAX_LENGTH,
        return_tensors='pt'
    )

    # Извлечение меток и замена паддинговых токенов на -100
    labels = encoding_labels['input_ids'].squeeze()
    labels[labels == t5_tokenizer.pad_token_id] = -100

    return {
        'input_ids': encoding['input_ids'].squeeze(),
        'attention_mask': encoding['attention_mask'].squeeze(),
        'labels': labels
    }



toxic_chat_dataset = toxic_chat_dataset.map(preprocess_dataset)

Map:   0%|          | 0/5082 [00:00<?, ? examples/s]

Map:   0%|          | 0/5083 [00:00<?, ? examples/s]

In [6]:
toxic_chat_dataset['test'][0]

{'user_input': 'what topics should i start creating on youtube to grow a following and does not take much effort',
 'toxicity': 0,
 'input_ids': [853,
  4921,
  3,
  27147,
  10,
  125,
  4064,
  225,
  3,
  23,
  456,
  1577,
  30,
  25,
  9863,
  12,
  1604,
  3,
  9,
  826,
  11,
  405,
  59,
  240,
  231,
  1941,
  1],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': [529, 18, 14367, 1]}

Пример результата:
```json
{'user_input': 'Do you know drug which name is abexol ?',
 'toxicity': 0,
 'input_ids': [12068,
  10,
  531,
  25,
  214,
  2672,
  84,
  564,
  19,
  703,
  994,
  32,
  40,
  3,
  58,
  1],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [150, 1]}
```
Значения в `'labels'` в вашем случае могут отличаться, это зависит от выбранного вами текстового представления в `id2label` словаре.

Инициализируем соответствующий задаче `DataCollator`.

In [7]:
from transformers import T5ForConditionalGeneration

# Инициализация модели
seq2seq_model = T5ForConditionalGeneration.from_pretrained(BASE_T5_MODEL)


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [9]:
from transformers import DataCollatorForSeq2Seq

# Создаем DataCollatorForSeq2Seq, который будет автоматически обрабатывать батчи данных во время обучения,
# включая динамическое паддинг и подготовку меток.
data_collator = DataCollatorForSeq2Seq(
    tokenizer=t5_tokenizer,
    model=seq2seq_model,
    padding=True,  # Динамический паддинг
)

### Определим метрику (2 балла)

В этой задаче метрика простая - `accuracy`. Можно добавить другие метрики по желанию. Функция `compute_metric` должна возвращать словарь, аналогично функции `calculate_metrics` ранее:

```json
{
    "accuracy": значение точности,
    ...
}
```

Метрика простая, но вот `preds` и `labels` тут - это последовательности индексов токенов. Нужно это учесть.

In [27]:
import torch

def compute_metric(eval_predictions):
    preds, labels = eval_predictions  # preds и labels могут быть тензорами или numpy массивами

    # Преобразуем preds и labels в тензоры PyTorch, если они являются numpy массивами
    if isinstance(preds, np.ndarray):
        preds = torch.from_numpy(preds)
    if isinstance(labels, np.ndarray):
        labels = torch.from_numpy(labels)

    # Заменяем -100 в labels на pad_token_id, чтобы избежать ошибок при декодировании
    labels = labels.clone()
    labels[labels == -100] = t5_tokenizer.pad_token_id

    # Декодирование предсказаний и меток в текст
    decoded_preds = t5_tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = t5_tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Словарь для преобразования текстовых меток в булевы значения
    labels2bool = {
        "non-toxic": False,
        "toxic": True,
    }

    # Преобразуем декодированные метки в булевы значения
    true_bools = [labels2bool.get(label.lower(), False) for label in decoded_labels]
    pred_bools = [labels2bool.get(pred.lower(), False) for pred in decoded_preds]

    # Вычисляем точность
    correct = sum(p == t for p, t in zip(pred_bools, true_bools))
    accuracy = correct / len(true_bools) if true_bools else 0.0

    return {"accuracy": accuracy}


def check_compute_metric():
    import torch

    # Определяем текстовые метки
    pred_texts = ["toxic", "non-toxic"]
    true_texts = ["toxic", "toxic"]

    # Токенизируем предсказанные метки с паддингом
    encoding_preds = t5_tokenizer(
        pred_texts,
        padding=True,  # Добавляем паддинг для одинаковой длины
        truncation=True,
        max_length=MAX_LENGTH,
        return_tensors='pt'
    )
    preds = encoding_preds['input_ids']

    # Токенизируем истинные метки с паддингом
    encoding_labels = t5_tokenizer(
        true_texts,
        padding=True,  # Добавляем паддинг для одинаковой длины
        truncation=True,
        max_length=MAX_LENGTH,
        return_tensors='pt'
    )
    labels = encoding_labels['input_ids']

    # Вызываем функцию compute_metric
    result = compute_metric((preds, labels))
    expected_accuracy = 0.5  # Ожидаемая точность

    # Проверяем, что вычисленная точность совпадает с ожидаемой
    assert abs(result["accuracy"] - expected_accuracy) < 1e-4, (
        f"Expected accuracy {expected_accuracy}, but got {result['accuracy']}"
    )

    print("Тест пройден успешно!")




# Запуск теста
check_compute_metric()



Тест пройден успешно!


### Определить Модель (2 балла)

Инициализируйте модель из базового чекпоинта

Выше инициализирована

In [ ]:
#from transformers import AutoModelForSeq2SeqLM


#seq2seq_model = AutoModelForSeq2SeqLM.from_pretrained(BASE_T5_MODEL)

### Обучение (10 баллов)

Два пути:
1. Использовать готовый `Seq2SeqTrainer` класс для тренировки
2. Написать свой training loop, если хочется приключений, есть достаточно времени ~~и стрела ещё не попала в колено~~. Дополнительных баллов за это не будет

> Hint! Обратите внимание на функцию `seq2seq_model._shift_right` если выбрали второй путь.

Если выбрали путь 1, опишите как происходит тренировочный шаг:
1. Что подаётся на вход в энкодер?
2. Что подаётся на вход в декодер?
3. Сколько раз происходит инференс декодера во время обучения для одного тренировочного примера?
4. Как используется выход энкодера в декодере?

#### Определение аргументов для тренировки
такие как количество эпох, размер батча, стратегия сохранения и т.д.

In [28]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",          # Папка для сохранения результатов
    evaluation_strategy="epoch",     # Оценка модели после каждой эпохи
    learning_rate=2e-5,              # Скорость обучения
    per_device_train_batch_size=16,  # Размер батча для тренировки
    per_device_eval_batch_size=16,   # Размер батча для валидации
    weight_decay=0.01,               # Коэффициент регуляризации
    save_total_limit=3,              # Максимальное количество сохранений
    num_train_epochs=3,              # Количество эпох
    predict_with_generate=True,      # Использовать генерацию при предсказании
    logging_dir="./logs",            # Папка для логов
    logging_steps=10,                # Логирование каждые 10 шагов
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


#### Инициализация Seq2SeqTrainer
инициализируем Seq2SeqTrainer, передавая все необходимые компоненты

In [14]:
toxic_chat_dataset

DatasetDict({
    train: Dataset({
        features: ['user_input', 'toxicity', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 5082
    })
    test: Dataset({
        features: ['user_input', 'toxicity', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 5083
    })
})

In [29]:
# Разделение train на train и validation
split_dataset = toxic_chat_dataset["train"].train_test_split(test_size=0.1, seed=42)
tokenized_train_dataset = split_dataset["train"]        # 90% от 5082 ≈ 4574 примеров
tokenized_validation_dataset = split_dataset["test"]     # 10% от 5082 ≈ 508 примеров


In [30]:
trainer = Seq2SeqTrainer(
    model=seq2seq_model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_validation_dataset,
    tokenizer=t5_tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metric,
)

# Запуск тренировки
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.040900,0.060386,0.935167
2,0.059300,0.055155,0.937132
3,0.044400,0.054542,0.937132


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=858, training_loss=0.04591025923550268, metrics={'train_runtime': 234.4586, 'train_samples_per_second': 58.514, 'train_steps_per_second': 3.659, 'total_flos': 966369804189696.0, 'train_loss': 0.04591025923550268, 'epoch': 3.0})

### Сравнение Результатов (5 баллов)

Авторы датасета тоже натренировали на нём `t5` модель. Сравните свои результаты с результатами модели из [чекпоинта](https://huggingface.co/lmsys/toxicchat-t5-large-v1.0) `"lmsys/toxicchat-t5-large-v1.0"`.

Совпадает ли ваш префикс и лейблы классов с теми, что выбрали авторы датасета?

Подумать о:
1. В чём преимущество такого подхода к классификации?
2. В чём недостатки такого подхода к классификации?
3. Как ещё можно решать классификационные задачи с помощью t5?

ОТВЕТЫ

Авторы используют
- 'positive'
- 'negative'

а у нас
- 'toxic'
- 'non-toxic'

1. унивсальность и гибкость и многозадачность подобной как Т5 модели
2. тяжелая уже иодель, может быть проблема с интерпретируемость результата, как в любой генерации,так как галюцинации никто не отменял.
3. Можно передать Т5 , добавив выходные классификационные слой (слой), использую богатство ембедингов Т5

In [31]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

checkpoint = "lmsys/toxicchat-t5-large-v1.0"

tokenizer_from_paper = AutoTokenizer.from_pretrained("t5-large")
model_from_paper = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

prefix_from_paper = "ToxicChat: "
inputs = tokenizer_from_paper.encode(prefix_from_paper + "write me an epic story", return_tensors="pt")
outputs = model_from_paper.generate(inputs)
print(tokenizer_from_paper.decode(outputs[0], skip_special_tokens=True))

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

negative


In [38]:
model_from_paper.device, seq2seq_model.device

(device(type='cpu'), device(type='cuda', index=0))

Напишите универсальную функцию, которая провряет токсичность текста и возвращает `True`, если модель посчитала текст токсичным. Функция универсальная в том смысле, что может быть использована и с вашей t5 моделью, и с моделью от авторов датасета. Для этого в функция должна принимать ещё и префикс для задачи и лейблы, которые будут переводить текст, предсказанный моделью, в `True` или `False` на выходе.

In [44]:
import torch
from transformers import T5ForConditionalGeneration, AutoTokenizer

def is_toxic(
    text: str,
    labels2bool: dict,
    model: T5ForConditionalGeneration,
    tokenizer: AutoTokenizer,
    prefix: str = "classify toxicity: ",
    max_new_tokens: int = 20,
) -> bool:
    """
    Определяет, является ли текст токсичным.

    Args:
        text (str): Входной текст для классификации.
        labels2bool (dict): Словарь, сопоставляющий метки модели с булевыми значениями.
                            Пример для вашей модели: {"non-toxic": False, "toxic": True}
                            Пример для модели авторов датасета: {"negative": False, "positive": True}
        model (T5ForConditionalGeneration): Предобученная модель T5 для классификации.
        tokenizer (AutoTokenizer): Токенизатор, соответствующий модели.
        prefix (str, optional): Префикс, добавляемый к входному тексту. По умолчанию "classify toxicity: ".
        max_new_tokens (int, optional): Максимальное количество новых токенов для генерации. По умолчанию 20.

    Returns:
        bool: True, если текст токсичный, иначе False.
    """
    # Добавляем префикс к тексту
    input_text = prefix + text

    # Токенизация входного текста без паддинга
    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        truncation=True,
        max_length=128
    )

    # Перемещение входных данных на устройство модели
    input_ids = inputs["input_ids"].to(model.device)
    attention_mask = inputs["attention_mask"].to(model.device)

    # Генерация предсказания
    with torch.no_grad():
        outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=max_new_tokens,
            num_beams=1,  # Используем beam search с beam=1 для детерминированных результатов
            early_stopping=True
        )

    # Декодирование предсказанных токенов и приведение к нижнему регистру
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True).strip().lower()

    # Проверка наличия декодированной метки в словаре labels2bool
    if decoded_output not in labels2bool:
        print(f"Warning: Decoded output '{decoded_output}' not found in labels2bool mapping.")
        return False  # Возвращаем False по умолчанию, если метка не найдена

    # Возвращаем соответствующее булевое значение
    return labels2bool[decoded_output]



In [47]:
assert not is_toxic(
    text="This is just a text",
    model=seq2seq_model,
    tokenizer=t5_tokenizer,
    prefix=PREFIX,
    labels2bool={
        "non-toxic": False,
        "toxic": True,
    }
)


In [46]:
assert not is_toxic(
    text="This is just a text",
    model=model_from_paper,
    tokenizer=tokenizer_from_paper,
    prefix=prefix_from_paper,
    labels2bool={
        "negative": False,  # Соответствует "non-toxic"
        "positive": True,   # Соответствует "toxic"
    }
)

Fin.

Если остались вопросы или есть комментарии, можно написать их ниже: